In [42]:
import numpy as np
import json
import os
import pandas as pd


In [43]:
#generate division dataset lets have digits upto 6 in a number
#for each digit have 500 samples 250 positive and 250 negative

def generate_division_dataset():
    
    data_pos = []
    data_neg = []
    #generate all 90 combinations of 1 digit numbers
    examples=0
    for i in range(1,10):
        for j in range(1,10):
            if i==j:
                continue
            div = i/j
            div_4 = round(div,4)
            div_8 = round(div,8)
            div_12 = round(div,12)
            data_pos.append([i,j,div_4,div_8,div_12])
            
            inv_div = j/i
            inv_div_4 = round(inv_div,4)
            inv_div_8 = round(inv_div,8)
            inv_div_12 = round(inv_div,12)
            
            data_pos.append([j,i,inv_div_4,inv_div_8,inv_div_12])
            
            
            
            #negative samples
            prob=np.random.rand()
            if prob<0.5:
                data_neg.append([-i,-j,div_4,div_8,div_12])
                data_neg.append([-j,-i,inv_div_4,inv_div_8,inv_div_12])
                data_neg.append([-i,j,-div_4,-div_8,-div_12])
                data_neg.append([j,-i,-inv_div_4,-inv_div_8,-inv_div_12])
                
                if examples > 37:
                    data_neg.append([i,-j,-div_4,-div_8,-div_12])
                    data_neg.append([-j,i,-inv_div_4,-inv_div_8,-inv_div_12])
                
                
            else:
                data_neg.append([-i,j,-div_4,-div_8,-div_12])
                data_neg.append([j,-i,-inv_div_4,-inv_div_8,-inv_div_12])
                data_neg.append([i,-j,-div_4,-div_8,-div_12])
                data_neg.append([-j,i,-inv_div_4,-inv_div_8,-inv_div_12])
                
                if examples > 37:
                    data_neg.append([-i,-j,div_4,div_8,div_12])
                    data_neg.append([-j,-i,inv_div_4,inv_div_8,inv_div_12])
                
            
            examples+=1
                
            
                
    len_1_digit_pos = len(data_pos)
    len_1_digit_neg = len(data_neg)
    
    #add to df_pos and df_neg
    df_pos = pd.DataFrame(data_pos,columns=['a','b','div_4','div_8','div_12'])
    df_neg = pd.DataFrame(data_neg,columns=['a','b','div_4','div_8','div_12'])
    
    #concat both dataframes
    df = pd.concat([df_pos,df_neg])
    


    for digit in range(2,7):
        i=0
        data_pos = []
        data_neg = []
        while i < 125:
            
            if i<70:
                a=np.random.randint(10**(digit-1),10**digit)
                b=np.random.randint(10**(digit-1),10**digit)
            

            else:
                a=np.random.randint(2,10**(digit-1))
                b=np.random.randint(10**(digit-1),10**digit)
            
            #calculate division in 3 types of precision
            #4,8,12
            i+=1
            if a==0 or b==0 or a==b:
                #goto sample generation again
                i-=1
                continue
            
            
            div = a/b
            div_4 = round(div,4)
            div_8 = round(div,8)
            div_12 = round(div,12)
            
            #check a b are previously generated or not
            #data is [a,b,div_4,div_8,div_12]
            if [a,b,div_4,div_8,div_12] in data_pos:
                i-=1
                continue
            
            inv_div = b/a
            inv_div_4 = round(inv_div,4)
            inv_div_8 = round(inv_div,8)
            inv_div_12 = round(inv_div,12)
            
 
            data_pos.append([a,b,div_4,div_8,div_12])
            data_pos.append([b,a,inv_div_4,inv_div_8,inv_div_12])
            
            
            #negative samples
            #0.33 prob a and b negative
            #0.33 prob a negative and b positive
            #0.33 prob a positive and b negative
            prob=np.random.rand()
            
            if prob<0.33:
                data_neg.append([-a,-b,div_4,div_8,div_12])
                data_neg.append([-b,-a,inv_div_4,inv_div_8,inv_div_12])
            elif prob<0.66:
                data_neg.append([-a,b,-div_4,-div_8,-div_12])
                data_neg.append([b,-a,-inv_div_4,-inv_div_8,-inv_div_12])
                
            else:
                data_neg.append([a,-b,-div_4,-div_8,-div_12])
                data_neg.append([-b,a,-inv_div_4,-inv_div_8,-inv_div_12])
                
        #add to df_pos and df_neg
        df_pos = pd.DataFrame(data_pos,columns=['a','b','div_4','div_8','div_12'])
        df_neg = pd.DataFrame(data_neg,columns=['a','b','div_4','div_8','div_12'])
        
        df_digit = pd.concat([df_pos,df_neg])
        
        #append to df
        df = pd.concat([df,df_digit])
    
    #save the data
    df.to_csv('division_dataset.csv',index=False)
    
    return df


generate_division_dataset()



,a,b,div_4,div_8,div_12
0,1,2,0.5000,0.500000,0.500000
1,2,1,2.0000,2.000000,2.000000
2,1,3,0.3333,0.333333,0.333333
3,3,1,3.0000,3.000000,3.000000
4,1,4,0.2500,0.250000,0.250000
...,...,...,...,...,...
245,-346722,4648,-74.5960,-74.595955,-74.595955
246,-78843,853217,-0.0924,-0.092407,-0.092407
247,853217,-78843,-10.8217,-10.821722,-10.821722
248,-13027,962344,-0.0135,-0.013537,-0.013537


In [44]:
#convert csv to json where each row is a list of 5 elements do not include header
def csv_to_json():
    df = pd.read_csv('division_dataset.csv')
    data = df.values.tolist()
    with open('division_dataset.json','w') as f:
        json.dump(data,f)

csv_to_json()

